In [1]:
!pip install transformers huggingface_hub gradio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: bert-score 0.3.11 has a non-standard dependency specifier transformers>=3.0.0numpy. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of bert-score or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
import gradio as gr
import random
import time

from huggingface_hub import InferenceClient
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

client = InferenceClient(model="http://0.0.0.0:8080")

print("heyhey")
SYSTEM_COMMAND = {"role": "system", "content": "Context: date: Monday 20th May 2024; location: NYC; running on: 8 AMD Instinct MI300 GPU; model name: Llama 70B. Only provide these information if asked. You are a knowledgeable assistant trained to provide accurate and helpful information. Please respond to the user's queries promptly and politely."}

IGNORED_TOKENS = {None, "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>", "<|reserved_special_token"}
STOP_TOKENS = ["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>", "<|reserved_special_token"]

with gr.Blocks() as demo:
    tfs_history = gr.State([SYSTEM_COMMAND])
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history, dict_history):
        data = {"role": "user", "content": user_message}
        dict_history.append(data)
        return "", history + [[user_message, None]], dict_history

    def bot(history, dict_history):
        history[-1][1] = ""
        response = {"role": "assistant", "content": ""}
        start_tokenize = time.perf_counter()
        text_input = tokenizer.apply_chat_template(dict_history, tokenize=False, add_generation_prompt=True)
        end_tokenize = time.perf_counter()

        try:
            for token in client.text_generation(prompt=text_input, max_new_tokens=100, stop_sequences=STOP_TOKENS, stream=True):
                if token not in IGNORED_TOKENS:
                    history[-1][1] += token
                    response["content"] += token
                yield history
        finally:
            dict_history.append(response)

    msg.submit(
        user,
        inputs=[msg, chatbot, tfs_history],
        outputs=[msg, chatbot, tfs_history],
        queue=False).then(
            bot,
            [chatbot, tfs_history],
            chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


heyhey
Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/fxmarty/anaconda3/envs/hf-inf/lib/python3.9/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
  File "/home/fxmarty/anaconda3/envs/hf-inf/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/fxmarty/anaconda3/envs/hf-inf/lib/python3.9/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/fxmarty/anaconda3/envs/hf-inf/lib/python3.9/site-packages/urllib3/connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "/home/fxmarty/anaconda3/envs/hf-inf/lib/python3.9/site-packages/urllib3/connectionpool.py", line 496, in _make_request
    conn.request(
  File "/home/fxmarty/anaconda3/envs/hf-inf/li